In [2]:
from fetch_data import (fetch_erp_data,post_erp_data,update_erp_data,delete_erp_data)
from smart_extractor import extract_first_record
from auto_mapper import update_mapping_file
from standardizer import standardize_response
from normalize import normalize_input_fields
import pandas as pd
import os
import xmlrpc.client
from cryptography.fernet import Fernet

from dotenv import load_dotenv
load_dotenv()

MAPPINGS_DIR = "mappings"

print("🔧 ERP Integration Menu")
print("1. Fetch + Map + Standardize ERP Data")
print("2. Post / Update / Delete ERP Record")
choice = input("Select mode (1 or 2): ").strip()

# === Block 1: Fetch + Map + Standardize ===
if choice == "1":
    erp_name = input("Enter ERP Name: ").strip().lower()

    # === Dynamically load access key ===
    if erp_name == "odoo":
        api_url = os.getenv("ODOO_URL")
        ODOO_DB = os.getenv("ODOO_DB")
        ODOO_USERNAME = os.getenv("ODOO_USERNAME")
        fernet = Fernet(os.getenv("FERNET_KEY"))
        ODOO_PASSWORD = fernet.decrypt(os.getenv("ODOO_PASSWORD_ENC").encode()).decode()
        access_key = f"{ODOO_DB}|{ODOO_USERNAME}|{ODOO_PASSWORD}"
    elif erp_name=='sap':
        api_url = os.getenv("SAP_URL")
        access_key = os.getenv("SAP_ACCESS_KEY")

    else:
        print('Unidentified ERP:',erp_name)
        exit()

    service_name = input('Enter Service Name: ').strip().lower()

    print('📡 Fetching data from ERP...')
    data = fetch_erp_data(api_url, access_key, erp_name, service_name)

    if not data:
        print('❌ Failed to retrieve data.')
    else:
        record = extract_first_record(data, erp_name)
        if not record:
            print('❌ Could not extract a valid data record.')
        else:
            print(f'📦 ERP Output Keys: {list(record.keys())}')
            update_mapping_file(service_name, erp_name, record)
            print('🔁 Standardizing output...')
            standardized = standardize_response(service_name, erp_name, record)
            print('✅ Standardized Output:')
            print(standardized)

# === Block 2: POST / UPDATE / DELETE ===
elif choice == "2":
    erp_name = input("Enter ERP Name: ").strip().lower()
    api_url = ODOO_URL
    service_name = input("Enter Service Name: ").strip().lower()
    action = input("Choose Action (post/update/delete): ").strip().lower()

    # 🔁 Map service name to actual Odoo model
    service_to_model = {
        "employee": "hr.employee",
        "employees": "hr.employee",
        "partner": "res.partner",
        "product": "product.product",
        "sale_order": "sale.order",
        "invoice": "account.move",
    }

    model_name = service_to_model.get(service_name)
    if not model_name:
        print(f"❌ Unknown service name '{service_name}'. Please check your input.")
        exit()

    # Load field mapping
    mapping_path = os.path.join(MAPPINGS_DIR, f"{service_name}.csv")
    if not os.path.exists(mapping_path):
        print(f"❌ Mapping file for service '{service_name}' not found.")
        exit()

    df = pd.read_csv(mapping_path)
    if erp_name not in df.columns:
        print(f"❌ ERP '{erp_name}' not found in mapping file.")
        exit()

    # Build common_name → erp_field mapping
    common_to_erp = {
        row["common_name"]: row[erp_name]
        for _, row in df.iterrows()
        if pd.notna(row["common_name"]) and pd.notna(row[erp_name])
    }

    # === Get writable fields from Odoo ===
    try:
        db, username, password = access_key.split("|")
        common = xmlrpc.client.ServerProxy(f"{api_url}/xmlrpc/2/common")
        uid = common.authenticate(db, username, password, {})
        models = xmlrpc.client.ServerProxy(f"{api_url}/xmlrpc/2/object")

        fields_meta = models.execute_kw(
            db, uid, password,
            model_name, 'fields_get',
            [], {'attributes': ['readonly']}
        )
        writable_fields = [k for k, meta in fields_meta.items() if not meta.get('readonly')]

        writable_common_names = [
            cname for cname, erp_field in common_to_erp.items() if erp_field in writable_fields
        ]

        print(f"📝 Writable fields for `{service_name}`:")
        for cname in writable_common_names:
            print(f" - {cname}")

    except Exception as e:
        print(f"⚠️ Could not fetch writable fields: {e}")
        writable_common_names = list(common_to_erp.keys())  # fallback to all fields

    # === Data input and submission ===
    if action == "post":
        print("Enter fields to POST (e.g. name=John). Type 'done' to finish:")
        user_input = {}
        while True:
            entry = input("> ").strip()
            if entry.lower() == "done":
                break
            if "=" in entry:
                key, value = entry.split("=", 1)
                user_input[key.strip()] = value.strip()

        erp_data = normalize_input_fields(user_input, common_to_erp)
        print("📡 Sending POST request to ERP...")
        post_erp_data(api_url, access_key, erp_name, model_name, erp_data)

    elif action == "update":
        record_id = int(input("Enter record ID to update: "))
        print("Enter fields to UPDATE (e.g. name=Jane). Type 'done' to finish:")
        user_input = {}
        while True:
            entry = input("> ").strip()
            if entry.lower() == "done":
                break
            if "=" in entry:
                key, value = entry.split("=", 1)
                user_input[key.strip()] = value.strip()

        erp_data = normalize_input_fields(user_input, common_to_erp)
        print("📡 Sending UPDATE request to ERP...")
        update_erp_data(api_url, access_key, erp_name, model_name, record_id, erp_data)

    elif action == "delete":
        record_id = int(input("Enter record ID to DELETE: "))
        print("📡 Sending DELETE request to ERP...")
        delete_erp_data(api_url, access_key, erp_name, model_name, record_id)

    else:
        print("❌ Invalid action selected.")

else:
    print("❌ Invalid mode selection. Please enter 1 or 2.")


🔧 ERP Integration Menu
1. Fetch + Map + Standardize ERP Data
2. Post / Update / Delete ERP Record


Select mode (1 or 2):  1
Enter ERP Name:  sap
Enter Service Name:  business


📡 Fetching data from ERP...
📦 ERP Output Keys: ['BusinessPartner', 'Customer', 'Supplier', 'AcademicTitle', 'AuthorizationGroup', 'BusinessPartnerCategory', 'BusinessPartnerFullName', 'BusinessPartnerGrouping', 'BusinessPartnerName', 'BusinessPartnerUUID', 'CorrespondenceLanguage', 'CreatedByUser', 'CreationDate', 'CreationTime', 'FirstName', 'FormOfAddress', 'Industry', 'InternationalLocationNumber1', 'InternationalLocationNumber2', 'IsFemale', 'IsMale', 'IsNaturalPerson', 'IsSexUnknown', 'GenderCodeName', 'Language', 'LastChangeDate', 'LastChangeTime', 'LastChangedByUser', 'LastName', 'LegalForm', 'OrganizationBPName1', 'OrganizationBPName2', 'OrganizationBPName3', 'OrganizationBPName4', 'OrganizationFoundationDate', 'OrganizationLiquidationDate', 'SearchTerm1', 'SearchTerm2', 'AdditionalLastName', 'BirthDate', 'BusinessPartnerBirthDateStatus', 'BusinessPartnerBirthplaceName', 'BusinessPartnerDeathDate', 'BusinessPartnerIsBlocked', 'BusinessPartnerType', 'ETag', 'GroupBusinessPartner